In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [13]:
## csv of s&p 500 data from yahoo finance
filepath = "../Input/^GSPC.csv"

# Load spreadsheet
xl = pd.ExcelFile("../Input/government_shutdown_stats.xlsx")

# Load a sheet into a DataFrame by name:
shutdown_df = xl.parse(xl.sheet_names[0])


In [14]:
## read stock csv
stock_prices_unclean = pd.read_csv(filepath)


In [15]:
# ## make the index the date to help with cleaning
stock_prices_unclean = stock_prices_unclean.set_index(['Date'])

In [27]:
stock_prices_unclean.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1976-09-15,103.940002,104.699997,103.279999,104.250000,104.250000,17570000
1976-09-16,104.250000,105.589996,103.839996,105.339996,105.339996,19620000
1976-09-17,105.339996,106.809998,105.139999,106.269997,106.269997,28270000
1976-09-20,106.269997,107.199997,105.739998,106.320000,106.320000,21730000
1976-09-21,106.320000,108.129997,106.089996,107.830002,107.830002,30300000


In [35]:
shutdown_df

,shutdown_number,from_date,to_date,length_in_days,pres_party_in_power,president,senate_party_in_power,senate_seats,house_party_in_power,house_seats
0,1,1976-09-30,1976-10-11,12,R,Ford,D,62 - 38,D,291 - 144
1,2,1977-09-30,1977-10-13,14,D,Carter,D,59 - 41,D,292 - 143
2,3,1977-10-31,1977-11-09,10,D,Carter,D,59 - 41,D,292 - 143
3,4,1977-11-30,1977-12-09,10,D,Carter,D,59 - 41,D,292 - 143
4,5,1978-09-30,1978-10-18,19,D,Carter,D,59 - 41,D,292 - 143
5,6,1979-09-30,1979-10-12,13,D,Carter,D,58 - 42,D,277 - 158
6,7,1981-11-20,1981-11-23,4,R,Reagan,R,53 - 47,D,244 - 191
7,8,1982-09-30,1982-10-02,3,R,Reagan,R,53 - 47,D,244 - 191
8,9,1982-12-17,1982-12-21,5,R,Reagan,R,53 - 47,D,244 - 191
9,10,1983-11-10,1983-11-14,5,R,Reagan,R,55 - 45,D,271 - 164


In [82]:
## loops through all of csv, only keeping date ranges we need
stock_prices_clean = pd.DataFrame()
obs_count_list = []
for shutdown in range(len(shutdown_df)):
    try: 
        from_date = stock_prices_unclean.index.get_loc(shutdown_df["from_date"][shutdown])
    except KeyError: 
        from_date = stock_prices_unclean.index.get_loc(str((pd.to_datetime(shutdown_df["from_date"][shutdown]) + datetime.timedelta(days=2)).date()))

    try: 
        to_date = stock_prices_unclean.index.get_loc(shutdown_df["to_date"][shutdown])
    except KeyError: 
        to_date = stock_prices_unclean.index.get_loc(str((pd.to_datetime(shutdown_df["to_date"][shutdown]) + datetime.timedelta(days=2)).date()))
        
    obs_count_list.append(len(stock_prices_unclean.iloc[from_date-3:to_date+3]))
    stock_prices_clean = stock_prices_clean.append(stock_prices_unclean.iloc[from_date-3:to_date+3])


In [83]:
obs_count_list

[13, 15, 13, 13, 18, 14, 7, 8, 8, 8, 7, 8, 8, 8, 8, 12, 28, 18, 6, 6]

In [88]:
## add shutdown number to the table
stock_prices_clean["Shutdown Number"] = 0
shutdown = 1
beg = 0
for x in obs_count_list:
    stock_prices_clean["Shutdown Number"].iloc[beg:(beg+x-1)] = shutdown
    shutdown += 1
    beg = (beg + x)

C:\Users\Amrit\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [89]:
stock_prices_clean

,Open,High,Low,Close,Adj Close,Volume,Shutdown Number
Date,,,,,,,
1976-09-27,106.800003,107.699997,106.349998,107.269997,107.269997,17430000,1
1976-09-28,107.269997,107.540001,105.610001,105.919998,105.919998,20440000,1
1976-09-29,105.919998,106.449997,104.830002,105.370003,105.370003,18090000,1
1976-09-30,105.370003,105.839996,104.570000,105.239998,105.239998,14700000,1
1976-10-01,105.239998,105.750000,103.599998,104.169998,104.169998,20620000,1
1976-10-04,104.169998,104.620003,103.419998,104.029999,104.029999,12630000,1
1976-10-05,104.029999,104.250000,102.510002,103.230003,103.230003,19200000,1
1976-10-06,103.230003,103.720001,102.050003,102.970001,102.970001,20870000,1
1976-10-07,102.970001,103.900002,102.160004,103.540001,103.540001,19830000,1
